In [1]:
import pickle
import torch
import pandas as pd
import plotly.graph_objects as go

c:\Users\parsa96\Anaconda3\envs\RL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
with open(f"Results/Toy-NECOM-lpless_single_1/Toy-NECOM-lpless_single_1_4000.pkl", 'rb') as f:
       agent1 = pickle.load(f)

In [67]:
obs_tens=torch.rand(1000,4)
obs_tens*=100
obs_tens[:,-1]=50

In [68]:
fluxes=torch.matmul(agent1.actor_network_(obs_tens),agent1.model.nullspace).detach().numpy()

In [69]:
obs_df=pd.DataFrame(obs_tens.detach().numpy(),columns=["agent1","Glc","Starch","time"])

In [70]:
fluxes_df=pd.DataFrame(fluxes,columns=[i.id for i in agent1.model.reactions])

In [71]:
full_table=pd.concat([obs_df,fluxes_df],axis=1)

In [75]:
fig = go.Figure(data=[go.Scatter3d(x=full_table["agent1"], y=full_table["Starch"], z=full_table["Amylase_Prod"],
                                   mode='markers',
                                   marker=dict(
                                    size=2,
                                    color=full_table["Amylase_e"],                # set color to an array/list of desired values
                                    colorscale='Viridis',   # choose a colorscale
                                    opacity=0.8))])
fig.show()

In [76]:
full_table.corr()

,agent1,Glc,Starch,time,Glc_e,ATP_Cat,ATP_M,X_Production,X_Ex,P_Prod,P_e,Amylase_Prod,Amylase_e
agent1,1.000000,-0.053707,0.001250,NaN,-0.537990,0.537990,0.537992,0.537989,0.537989,0.537990,0.537990,0.537995,0.537995
Glc,-0.053707,1.000000,-0.044904,NaN,-0.769292,0.769292,0.769290,0.769293,0.769293,0.769292,0.769292,0.769288,0.769288
Starch,0.001250,-0.044904,1.000000,NaN,-0.225121,0.225121,0.225122,0.225121,0.225121,0.225121,0.225121,0.225124,0.225124
time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glc_e,-0.537990,-0.769292,-0.225121,NaN,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
ATP_Cat,0.537990,0.769292,0.225121,NaN,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
ATP_M,0.537992,0.769290,0.225122,NaN,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
X_Production,0.537989,0.769293,0.225121,NaN,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
X_Ex,0.537989,0.769293,0.225121,NaN,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
P_Prod,0.537990,0.769292,0.225121,NaN,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
